In [5]:
from langchain.chat_models import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

key = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo" # 모델 이름
    #, api_key=key # API 키
    , temperature=0.1 # 창의력
  )


In [51]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean."),
    AIMessage(content="안녕하세요. 제 이름은 쳇! 입니다."),
    HumanMessage(content="What is the distance between korea and japan?, What is your name?")
]

chat.predict_messages(messages)

AIMessage(content='한국과 일본 사이의 거리는 약 800km입니다. 저의 이름은 쳇! 입니다. 어떻게 도와드릴까요?')

In [52]:
# PromptTemplate 사용
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the distance between {country1} and {country2}")
prompt = template.format(country1 = "Korea", country2 = "japan")

prompt = template.format(
  country1 = "Korea",
  country2 = "Japan"
)

chat.predict(prompt)

'The distance between Korea and Japan varies depending on the specific locations being measured. On average, the distance between Seoul, South Korea and Tokyo, Japan is approximately 1,100 kilometers (683 miles) when measured in a straight line. However, the actual distance can be longer when taking into account the specific points of departure and arrival, as well as the mode of transportation being used.'

In [6]:
# ChatPromptTemplate 사용
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕하세요. 제 이름은 {name} 입니다."),
    ("human", "What is the distance between {country1} and {country2}?, Also, what is your name?")
  ]
)

prompt = template.format_messages(
  language="Korean"
  , name="쳇!"
  , country1="Korea"
  , country2="Japan"
)

chat.predict_messages(prompt)

AIMessage(content='한국과 일본 사이의 거리는 약 800km입니다. 제 이름은 쳇! 입니다.')

In [9]:
# OutputParser 사용
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",") # 앞 뒤 공백을 제거 후 "," 로 분리
        return list(map(str.strip, items))
      
p = CommaOutputParser()

p.parse("Heelow, how, are, you")

['Heelow', 'how', 'are', 'you']

In [18]:
# LangChain expression language 사용
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are list generatting machine. Everything you are asked " + 
      "will be answerd with a comma separated list of max {max_items}." + 
      "Do not reply with anything else."),
    ("human", "{question}"),
  ]
)

# chain 으로 Chat, ChatPromptTemplate, OutputParser를 한 번에 작성 할 수 있다.
chain = template | chat | CommaOutputParser()
chain.invoke({
  "max_items": 10,
  "question": "What are the Pocketmons?"
})


['Pikachu',
 'Charmander',
 'Bulbasaur',
 'Squirtle',
 'Jigglypuff',
 'Eevee',
 'Snorlax',
 'Mewtwo',
 'Gengar',
 'Lucario']

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "Your are world-class international chef. you create easy to follow recipies for any type of cusine with easy to find ingredients."),
    ("human", "I want to {cuisine} food.")
  ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
  [
    ("system" "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
  ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "indian"
})

One of my favorite Indian dishes to make is Chicken Tikka Masala. It's a flavorful and creamy dish that is sure to impress your friends and family. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp paprika
- 1 tsp ground cinnamon
- 1 tsp cayenne pepper
- 1 tsp ground black pepper
- 1 tsp salt
- 1 tbsp grated fresh ginger
- 3 cloves garlic, minced
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- 2 tbsp vegetable oil
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a large bowl, combine the yogurt, lemon juice, cumin, paprika, cinnamon, cayenne pepper, black pepper, salt, ginger, and garlic. Add the chicken pieces and toss to coat. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. In a large skillet, heat the vegetable oil over medium-high heat. Add the ma

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based protein such as tofu or seitan. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb firm tofu or seitan, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt for a vegan option)\n- 2 tbsp lemon juice\n- 2 tsp ground cumin\n- 2 tsp paprika\n- 1 tsp ground cinnamon\n- 1 tsp cayenne pepper\n- 1 tsp ground black pepper\n- 1 tsp salt\n- 1 tbsp grated fresh ginger\n- 3 cloves garlic, minced\n- 1 can (14 oz) tomato sauce\n- 1 cup coconut cream (or dairy-free heavy cream alternative)\n- 2 tbsp vegetable oil\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. In a large bowl, combine the yogurt, lemon juice, cumin, paprika, cinnamon, cayenne pepper, black pepper, salt, ginger, and garlic. Add the tofu or seitan pieces and toss to coat. Cover and refrigerate for at least 1 hour, or overnight 

: 

In [2]:
# PromptTemplate 사용 
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

# 아래 주석된 코드와 동일한 코드이다.
t = PromptTemplate.from_template("What is the capital of {country}")

# t = PromptTemplate(
#   template="What is the capital of {country}?",
#   input_variables=["country"],
# )

t.format(country="France")

'What is the capital of France'

: 

In [8]:
# FewShotPromptTemplate 사용, FewShot은 모델에서 예시를 준다는 뜻
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

# 우리가 원하는 답의 형태
examples = [
  {
    "question": "What do you know about France?",
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
    """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
    """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
    """,
  },
]

# 형식화 하기
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

few_shot_prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: What do you know about {country}?", # 마지막에 내용 포함
  input_variables=["country"], # 입력받을 값 지정
)

# few_shot_prompt.format(country="Korea")

chain = few_shot_prompt | chat

chain.invoke({
  "country": "Korea"
})

AI: 
      Here is what I know:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won

AIMessageChunk(content='AI: \n      Here is what I know:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won')

In [7]:
# FewShotChatMessagePromptTemplate 사용
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

# 우리가 원하는 답의 형태
examples = [
  {
      "country": "France",
      "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
      "country": "Italy",
      "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
      "country": "Greece",
      "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]

# 형식화 하기
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
  ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})


      I know this:
      South Korea:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won

      North Korea:
      Capital: Pyongyang
      Language: Korean
      Currency: North Korean Won

AIMessageChunk(content='\n      I know this:\n      South Korea:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won\n\n      North Korea:\n      Capital: Pyongyang\n      Language: Korean\n      Currency: North Korean Won')

In [18]:
# dynamic 하게 example 을 선택 및 BaseExmapleSelector 사용 법
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

# 우리가 원하는 답의 형태
examples = [
  {
    "question": "What do you know about France?",
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
    """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
    """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
    """,
  },
]

class RandomExampleSelector(BaseExampleSelector):
  
  def __init__(self, examples):
    self.examples = examples
    
  def add_example(self, example):
    self.examples.append(example)
  
  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]

# 형식화 하기
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?", # 마지막에 내용 포함
  input_variables=["country"], # 입력받을 값 지정
)

prompt.format(country="France")

'Human: What do you know about Greece?\nAI: \n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n    \n\nHuman: What do you know about France?'

In [23]:
# 프롬프트를 import 하는 방법, json, yaml
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [25]:
# 프롬프트들의 memory 등을 다 모으는 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  temperature=0.1
  , streaming=True
  , callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrrrg!

AIMessageChunk(content="Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrrrg!")

In [35]:
# cash, debug, sql
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(False)

chat = ChatOpenAI(
  temperature=0.1
  # , streaming=True
  # , callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict("How do you make italian pasta.")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [38]:
# 비용 측정 방법, 모델 저장 및 불러오기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
  temperature=0.1
)

with get_openai_callback() as usage:
  a = chat.predict("What is the recipe for beer.")
  b = chat.predict("What is the recipe for bread.")
  print(a, b, "\n")
  print(usage)

Beer is typically made using the following ingredients and steps:

Ingredients:
- Water
- Barley malt
- Hops
- Yeast

Steps:
1. Malting: Barley grains are soaked in water and allowed to germinate. The germinated barley is then dried in a kiln to stop the germination process, creating malted barley.

2. Mashing: The malted barley is crushed and mixed with hot water in a process called mashing. This helps to extract the sugars from the barley.

3. Boiling: The liquid extracted from the mashing process, known as wort, is boiled and hops are added for flavor and aroma. The boiling process also helps to sterilize the wort.

4. Fermentation: The boiled wort is cooled and yeast is added. The yeast consumes the sugars in the wort and produces alcohol and carbon dioxide as byproducts. This process typically takes 1-2 weeks.

5. Conditioning: After fermentation is complete, the beer is conditioned for a period of time to allow flavors to develop and the beer to mature.

6. Packaging: The beer is